# Connecting Arduino to serial port via Pyserial

In [ ]:
#Import the necessary libraries
from time import sleep
import serial
import re
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

%matplotlib inline
#%load_ext autoreload
#%autoreload 2
%matplotlib notebook

## Example 1
Print something to the arduino and get it to send it back one bit at a time

In [ ]:
ser = serial.Serial('COM5', 9600) # Establish the connection on a specific port

In [ ]:
print(ser.name)         # check which port was really used

In [ ]:
ser.write(b'hello') # Convert the decimal number to ASCII then send it to the Arduino
sleep(.1) #  This delay is necessary for the input to get updated
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second

In [ ]:
ser.close() #close the serial communication

## Example 2
Print a string to the arduino and get the whole string back

In [ ]:
ser = serial.Serial('COM5', 9600)
print(ser.name)
initial = ""
sleep(1)
while ("Ready" not in str(initial)):
    initial = ser.readline()
print(initial)

In [ ]:

ser.write(b'hello') # Convert the decimal number to ASCII then send it to the Arduino
sleep(2) #  This delay is necessary for the input to get updated, I could not figure how long is exactly necessary
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second


In [ ]:
ser.close()

## Example 3

a) Turning an LED on and off
b) The above but employ "serial event" in the code

Note that serial event is an interupt and using this allows us to free up our main loop and not check the serial buffer at every instance. You will soon see why it is useful.

In [ ]:
ser = serial.Serial('COM5', 9600)
print(ser.name)
initial = ""
sleep(1)
while ("Ready" not in str(initial)):
    initial = ser.readline()
print(initial)

In [ ]:
ser.write(b'OFF') 
sleep(2) #  This delay is necessary for the input to get updated, I could not figure how long is exactly necessary
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second


In [ ]:
ser.write(b'ON') # Convert the decimal number to ASCII then send it to the Arduino


In [ ]:
ser.close()

## Example 4

Let's make things a bit more advanced. We will now send ON/OFF commands but when we sent the ON command we also send an intensity between 0-255. This will be similar to Arduino's own dimmer example.

The commands will be:

- ON #A_number

- OFF

In [ ]:
ser = serial.Serial('COM5', 9600)
print(ser.name)
initial = ""
sleep(1)
while ("Ready" not in str(initial)):
    initial = ser.readline()
print(initial)

In [ ]:
ser.write(b'ON 100') 
sleep(2) #  This delay is necessary for the input to get updated, I could not figure how long is exactly necessary
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second
ser.write(b'ON 250') 
sleep(2) #  This delay is necessary for the input to get updated, I could not figure how long is exactly necessary
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second
ser.write(b'ON 50') 
sleep(2) #  This delay is necessary for the input to get updated, I could not figure how long is exactly necessary
while ser.in_waiting:
    print (ser.readline()) # Read the newest output from the Arduino
    sleep(.1) # Delay for one tenth of a second
ser.write(b'OFF')

In [ ]:
ser.close()

## Example 5

Let's now hook up a system to read one of the analog pins (perhaps an analog sensor) every seconds and plot the realtime data here.

In [ ]:
ser = serial.Serial('COM5', 9600)
ser.timeout = 10 #specify timeout when using readline()
#ser.open()
if ser.is_open==True:
	print("\nAll right, serial port now open. Configuration:\n")
	print(ser, "\n")
sleep(1)

In [ ]:
fig = plt.figure(figsize=(6,6))
#ax = fig.add_subplot(111)
plt.ion()
count = 0
xs =[]
ys = []

fig.show()
plt.plot(ys,'-or')
plt.yticks(np.arange(-200, 1100, 50.0))
plt.tight_layout()
fig.canvas.draw()
#iterate - plot, erase
while ser.in_waiting and count < 100:
    #print i
    count += 1
    xs.append(count)
    ys.append(int(re.findall('\d+', str(ser.readline()))[0]))
    plt.ylim(bottom=0)
    
    plt.plot(xs, ys,'-or')
    plt.tight_layout()
    plt.yticks(np.arange(-200, 1100, 50.0))
    fig.canvas.draw()   # draw
    sleep(1)    #sleep
#plt.plot(xs, ys,'-or')
#fig.show()

In [ ]:
ser.close()

## Example 6

We can put everything together and make a control and sensing station that can be managed via our notebook

Here's our list of 4 commands:

- LED ON/OFF : turn led on and off
- READ A1/A2 : read analogpin A1/A2 and send the results
- SERVO 1/2 #deg : turn servos 1 or 2 to the desired angle
- PWM 3/4 #duty : Put a PWM on pins 3 or 4


In [ ]:
ser = serial.Serial('COM5', 9600)
print(ser.name)
initial = ""
sleep(1)
while ("Ready" not in str(initial)):
    initial = ser.readline()
print(initial)

In [ ]:
ser.write(b'LED ON') 
print(ser.readline())

In [ ]:
ser.write(b'READ A2') 
print(ser.readline())
print(ser.readline())

In [ ]:
ser.write(b'READ A1') 
print(ser.readline())
print(ser.readline())

In [ ]:
ser.write(b'PWM 3 100') 
print(ser.readline())